In [ ]:
path_to_src = '/content/drive/MyDrive/Colab Notebooks/GNN/MoTSE-main/src'
dataset_name = "QM9"
task_lst = ["alpha","mu","homo","lumo","gap","r2","zpve","u0","u298","h298","g298","cv"]
dataset_metrics = ['r2','mae']
"""dataset_name = "PCBA"
task_lst = ["PCBA-1030","PCBA-1458","PCBA-1460","PCBA-2546","PCBA-2551","PCBA-485297","PCBA-485313","PCBA-485364","PCBA-504332","PCBA-504333","PCBA-504339","PCBA-504444","PCBA-504467","PCBA-588342","PCBA-624296","PCBA-624297","PCBA-624417","PCBA-651965","PCBA-652104","PCBA-686970","PCBA-686978","PCBA-686979","PCBA-720504"]
dataset_metrics = ['roc_auc','auprc']"""

#If using Google drive
from google.colab import drive
drive.mount('/content/drive')
# get the current working directory
import sys
sys.path.insert(0,path_to_src)

Mounted at /content/drive


In [ ]:
#!pip3 uninstall dgl -y
!pip3 install  dgl -f https://data.dgl.ai/wheels/cu118/repo.html
!pip3 install dgllife
!pip3 install rdkit

Looking in links: https://data.dgl.ai/wheels/cu118/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.8/93.8 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.1/226.1 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 780.9 kB/s eta 0:00:00


## Before moving on, make sure the "Calculate Similarity" section of motse.ipynb has been run for the dataset of interest

In [ ]:
import torch
import pandas as pd
from utils import load_model, makedir, set_random_seed
from utils.data import load_data
from multi_trainer import Multi_Trainer
from trainer import Trainer

%load_ext autoreload
%autoreload 2
set_random_seed(18) #It's been run on 22 for both

task_lst = task_lst[:10] #Only testing transfer to first 10 tasks

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#Pick best 3 source tasks for each target task based on MSE similarity

data_path = path_to_src + f'/../datasets/{dataset_name}/1000/'
model_type = 'GCN'
first_train_model_path = path_to_src + f"/../saved_models/{dataset_name}/GCN/raw_multi1000-1/" #To save model trained in tandem w/ first source task
second_train_model_path = path_to_src + f"/../saved_models/{dataset_name}/GCN/raw_multi1000-2/" #To save model trained in tandem w/ second source task
third_train_model_path = path_to_src + f"/../saved_models/{dataset_name}/GCN/raw_multi1000-3/" #To save model trained in tandem w/ third source task
first_ft_model_path = path_to_src + f"/../saved_models/{dataset_name}/GCN/multi1000-1/" #To save finetuned model trained in tandem w/ first source task
second_ft_model_path = path_to_src + f"/../saved_models/{dataset_name}/GCN/multi1000-2/" #To save finetuned model trained in tandem w/ second source task
third_ft_model_path = path_to_src + f"/../saved_models/{dataset_name}/GCN/multi1000-3/" #To save finetuned model trained in tandem w/ third source task
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
results_dict = dict()

source_target_pairs = []
similarity_results_path = path_to_src + f"/../results/{dataset_name}/GCN/10000/zinc500/"
for target_task in task_lst:
    similarity_df = pd.read_csv(similarity_results_path + target_task + '.csv')
    similarity_df = similarity_df.sort_values(by='motse', ascending=False)
    source_task1 = similarity_df.iloc[[1]]['source_task'].to_list()[0]
    source_task2 = similarity_df.iloc[[2]]['source_task'].to_list()[0]
    source_task3 = similarity_df.iloc[[3]]['source_task'].to_list()[0]
    source_target_pairs.append((source_task1, source_task2, source_task3, target_task))

In [ ]:
def multi_train(source_target_pairs, source_number, source_model_path, save_model_path):
  makedir(save_model_path)
  device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
  results_df = pd.DataFrame(columns = ['task'] + dataset_metrics)
  for source_task1, source_task2, source_task3, target_task in source_target_pairs:
    if source_number == 1:
      source_task = source_task1
      prev_source_task = ''
    elif source_number == 2:
      source_task = source_task2
      prev_source_task = source_task1
    else:
      source_task = source_task3
      prev_source_task = source_task2
    #Train w/ first source task
    print('multi ' + target_task + ' ' + source_task)
    train_loader1, val_loader1, test_loader1, data_args1 = load_data(
        dataset=dataset_name,
        data_path=data_path,
        tasks=[target_task],
        device = device
    )
    train_loader2, val_loader2, test_loader2, data_args2 = load_data(
        dataset=dataset_name,
        data_path=data_path,
        tasks=[source_task],
        device = device
    )
    if source_model_path is None:
      model = load_model(device=device, is_multi = True)
    else:
      model = load_model(device=device, is_multi = True, source_model_path = source_model_path+f"{target_task}.pth")
    trainer = Multi_Trainer(device=device,task1=target_task,task2=source_task,
                      data_args1=data_args1,data_args2=data_args2,model_path=save_model_path,
                     )
    model, task_results1_dict, task_results2_dict = trainer.fit(model, train_loader1,
                                      val_loader1, test_loader1, train_loader2, val_loader2, test_loader2)
    result = {'task': target_task}
    for metric in data_args1['metrics']:
        if metric in dataset_metrics:
          result[metric] = task_results1_dict[metric][target_task]
    results_df = results_df.append(result, ignore_index=True)
    result_path = save_model_path.replace('saved_models','results')
    makedir(result_path)
    results_df.to_csv(result_path+'results.csv', float_format='%.3f',index=False)
  print(f"Results have been saved to {result_path+'results.csv'}")

def multi_ft(source_target_pairs, source_number, source_model_path, save_model_path):
  makedir(save_model_path)
  device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
  if source_number == 1:
    results_df = pd.DataFrame(columns = ['task', 'source1'] + dataset_metrics)
  elif source_number == 2:
    results_df = pd.DataFrame(columns = ['task', 'source1', 'source2'] + dataset_metrics)
  else:
    results_df = pd.DataFrame(columns = ['task', 'source1', 'source2', 'source3'] + dataset_metrics)
  for source_task1, source_task2, source_task3, target_task in source_target_pairs:
      if source_number == 1:
        source_task = source_task1
        result = {'task': target_task, 'source1': source_task1}
      elif source_number == 2:
        source_task = source_task2
        result = {'task': target_task, 'source1': source_task1, 'source2': source_task2}
      else:
        source_task = source_task3
        result = {'task': target_task, 'source1': source_task1, 'source2': source_task2, 'source3': source_task3}
      print(f"ft on {target_task}")

      train_loader, val_loader, test_loader, data_args = load_data(
          dataset=dataset_name,data_path=data_path,tasks=[target_task],
          device = device
      )
      model = load_model(
          device=device,
          source_model_path=source_model_path+f"{target_task}_{source_task}.pth",
          is_multi = True
      )
      trainer = Trainer(
          device=device,tasks=[target_task],
          data_args=data_args,model_path=save_model_path)
      _, task_results_dict = trainer.fit(model, train_loader, val_loader, test_loader)
      for metric in data_args['metrics']:
          if metric in dataset_metrics:
            result[metric] = task_results_dict[metric][target_task]
      results_df = results_df.append(result, ignore_index=True)
      result_path = save_model_path.replace('saved_models','results')
      makedir(result_path)
      results_df.to_csv(result_path+'results.csv', float_format='%.3f',index=False)
  print(f"Results have been saved to {result_path+'results.csv'}")

In [ ]:
#Pre-train GNN w/ target task and first source task
#Use target data to quickly fine-tune model pre-trained on one source task + target (for comparison purposes w/ using 2 and 3 tasks)

multi_train(source_target_pairs, 1, None, first_train_model_path)
multi_ft(source_target_pairs, 1, first_train_model_path, first_ft_model_path)

multi PCBA-1030 PCBA-485364
819 loaded!
100 loaded!
980 loaded!
845 loaded!
95 loaded!
943 loaded!
initializing trainer...
[0] training loss:1.3818903951083912
PCBA-1030
val roc_auc:0.5818327330932371
val auprc:0.5689890575444164
PCBA-485364
val roc_auc:0.5957446808510638
val auprc:0.5438441117097391
[20] training loss:1.320008679932239
PCBA-1030
val roc_auc:0.6670668267306923
val auprc:0.6947261727743712
PCBA-485364
val roc_auc:0.5851063829787234
val auprc:0.6101672952576696
[40] training loss:1.273141814213173
PCBA-1030
val roc_auc:0.680672268907563
val auprc:0.7026312136523343
PCBA-485364
val roc_auc:0.5842198581560284
val auprc:0.6387145899212779
[60] training loss:1.2291815386098974
PCBA-1030
val roc_auc:0.6974789915966386
val auprc:0.7063302571611972
PCBA-485364
val roc_auc:0.600177304964539
val auprc:0.6356101129220623
test roc_auc:0.6076809330417579
test auprc:0.5777560752782758
test roc_auc:0.6305310132618358
test auprc:0.6300791359482817
multi PCBA-1458 PCBA-485297


<ipython-input-20-2d08720ff282>:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(result, ignore_index=True)


824 loaded!
92 loaded!
976 loaded!
822 loaded!
92 loaded!
1019 loaded!
initializing trainer...
[0] training loss:1.3797941932491227
PCBA-1458
val roc_auc:0.7603969754253308
val auprc:0.6989222599998243
PCBA-485297
val roc_auc:0.7069943289224953
val auprc:0.692963298114733
[20] training loss:1.223115024613399
PCBA-1458
val roc_auc:0.7958412098298677
val auprc:0.7403055039248935
PCBA-485297
val roc_auc:0.7637051039697542
val auprc:0.7284218930437961
[40] training loss:1.1758451356607325
PCBA-1458
val roc_auc:0.8171077504725898
val auprc:0.7604126178813101
PCBA-485297
val roc_auc:0.7622873345935727
val auprc:0.7076510426584635
[60] training loss:1.1469657631481396
PCBA-1458
val roc_auc:0.832703213610586
val auprc:0.7821583339978417
PCBA-485297
val roc_auc:0.772211720226843
val auprc:0.7032085247543527
[80] training loss:1.1391260425249736
PCBA-1458
val roc_auc:0.8511342155009453
val auprc:0.7999594952500467
PCBA-485297
val roc_auc:0.7750472589792061
val auprc:0.712953214731408
[100] train

<ipython-input-20-2d08720ff282>:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(result, ignore_index=True)


816 loaded!
96 loaded!
970 loaded!
810 loaded!
99 loaded!
983 loaded!
initializing trainer...
[0] training loss:1.3608676767349244
PCBA-1460
val roc_auc:0.575599128540305
val auprc:0.5839860213429147
PCBA-504332
val roc_auc:0.5073649754500819
val auprc:0.5384245253028699
[20] training loss:1.219774305820465
PCBA-1460
val roc_auc:0.6997821350762528
val auprc:0.7203287304730479
PCBA-504332
val roc_auc:0.5691489361702128
val auprc:0.5717201446124371
[40] training loss:1.167883129119873
PCBA-1460
val roc_auc:0.7045751633986929
val auprc:0.7336304885366839
PCBA-504332
val roc_auc:0.5765139116202945
val auprc:0.6095023862078861
[60] training loss:1.1303639018535614
PCBA-1460
val roc_auc:0.7612200435729847
val auprc:0.7886244993071321
PCBA-504332
val roc_auc:0.5495090016366613
val auprc:0.5999366581815604
test roc_auc:0.7001716012703616
test auprc:0.7065948080113519
test roc_auc:0.6265292261308198
test auprc:0.5891548674350799
multi PCBA-2546 PCBA-2551


...

<ipython-input-20-2d08720ff282>:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(result, ignore_index=True)


810 loaded!
99 loaded!
983 loaded!
loading pretrained model: /content/drive/MyDrive/Colab Notebooks/GNN/MoTSE-main/src/../saved_models/PCBA/GCN/raw_multi1000-1/PCBA-504332_PCBA-1030.pth
[0] training loss:0.6194121092557907
val roc_auc:0.5552373158756138
val auprc:0.576981972894156
[20] training loss:0.6030293172597885
val roc_auc:0.5605564648117839
val auprc:0.5817977503406658
[40] training loss:0.5819414484500885
val roc_auc:0.5519639934533551
val auprc:0.568117664601298
[60] training loss:0.5587746399641037
val roc_auc:0.5969721767594108
val auprc:0.6012980399323538
[80] training loss:0.543985190987587
val roc_auc:0.5908346972176759
val auprc:0.5885634893232742
test roc_auc:0.61123489410342
test auprc:0.5673890007216491
ft on PCBA-504333


<ipython-input-20-2d08720ff282>:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(result, ignore_index=True)


770 loaded!
91 loaded!
954 loaded!
loading pretrained model: /content/drive/MyDrive/Colab Notebooks/GNN/MoTSE-main/src/../saved_models/PCBA/GCN/raw_multi1000-1/PCBA-504333_PCBA-2551.pth
[0] training loss:0.6166895894954602
val roc_auc:0.6117073170731707
val auprc:0.6496691647051229
[20] training loss:0.5922591462731361
val roc_auc:0.6302439024390244
val auprc:0.6658319801998137
test roc_auc:0.6426630912607703
test auprc:0.646759760187538
Results have been saved to /content/drive/MyDrive/Colab Notebooks/GNN/MoTSE-main/src/../results/PCBA/GCN/multi1000-1/results.csv


<ipython-input-20-2d08720ff282>:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(result, ignore_index=True)


In [ ]:
#Pre-train GNN w/ target task and second source task, using pre-trained weights from target task + first source task
#Use target data to fine-tune model pre-trained on 2 source tasks + target

multi_train(source_target_pairs, 2, first_ft_model_path, second_train_model_path)
multi_ft(source_target_pairs, 2, second_train_model_path, second_ft_model_path)

multi PCBA-1030 PCBA-504332
819 loaded!
100 loaded!
980 loaded!
810 loaded!
99 loaded!
983 loaded!
loading pretrained model: /content/drive/MyDrive/Colab Notebooks/GNN/MoTSE-main/src/../saved_models/PCBA/GCN/multi1000-1/PCBA-1030.pth
initializing trainer...
[0] training loss:1.32080721616745
PCBA-1030
val roc_auc:0.6822729091636653
val auprc:0.6755977109163193
PCBA-504332
val roc_auc:0.5286415711947627
val auprc:0.5698408506316409
[20] training loss:1.2143967354297638
PCBA-1030
val roc_auc:0.6890756302521008
val auprc:0.7050786185847218
PCBA-504332
val roc_auc:0.569558101472995
val auprc:0.583725142218243
[40] training loss:1.1918754053115845
PCBA-1030
val roc_auc:0.6862745098039216
val auprc:0.6885701881810504
PCBA-504332
val roc_auc:0.5711947626841244
val auprc:0.5834764283679372
test roc_auc:0.6107716338644174
test auprc:0.5739600756975776
test roc_auc:0.6400096081370983
test auprc:0.6081580675993681
multi PCBA-1458 PCBA-485313


<ipython-input-20-2d08720ff282>:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(result, ignore_index=True)


824 loaded!
92 loaded!
976 loaded!
767 loaded!
111 loaded!
960 loaded!
loading pretrained model: /content/drive/MyDrive/Colab Notebooks/GNN/MoTSE-main/src/../saved_models/PCBA/GCN/multi1000-1/PCBA-1458.pth
initializing trainer...
[0] training loss:1.0377586255682276
PCBA-1458
val roc_auc:0.8889413988657845
val auprc:0.8771776904872397
PCBA-485313
val roc_auc:0.760697827518104
val auprc:0.7062966322953285
[20] training loss:1.0063664038130578
PCBA-1458
val roc_auc:0.8804347826086957
val auprc:0.8571088950810211
PCBA-485313
val roc_auc:0.7778143515470705
val auprc:0.7412054869993608
[40] training loss:0.9967681245600923
PCBA-1458
val roc_auc:0.8846880907372401
val auprc:0.8652882911253079
PCBA-485313
val roc_auc:0.7597103357472021
val auprc:0.7001082530118916
test roc_auc:0.7494775867880014
test auprc:0.686954827873821
test roc_auc:0.7576697241331387
test auprc:0.7412525073148604
multi PCBA-1460 PCBA-504333


<ipython-input-20-2d08720ff282>:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(result, ignore_index=True)


816 loaded!
96 loaded!
970 loaded!
770 loaded!
91 loaded!
954 loaded!
loading pretrained model: /content/drive/MyDrive/Colab Notebooks/GNN/MoTSE-main/src/../saved_models/PCBA/GCN/multi1000-1/PCBA-1460.pth
initializing trainer...
[0] training loss:1.212786336739858
PCBA-1460
val roc_auc:0.7664488017429194
val auprc:0.7899122027028023
PCBA-504333
val roc_auc:0.6858536585365853
val auprc:0.7360740044645173
[20] training loss:1.1395135534306366
PCBA-1460
val roc_auc:0.7686274509803921
val auprc:0.7971461422056192
PCBA-504333
val roc_auc:0.6834146341463415
val auprc:0.7284413483974348
[40] training loss:1.1114414843420188
PCBA-1460
val roc_auc:0.7742919389978213
val auprc:0.8086223741506511
PCBA-504333
val roc_auc:0.7141463414634146
val auprc:0.751922027938533
test roc_auc:0.7169176655397329
test auprc:0.715167989094303
test roc_auc:0.628112361526288
test auprc:0.6421785400248353
multi PCBA-2546 PCBA-1458


...

<ipython-input-20-2d08720ff282>:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(result, ignore_index=True)


810 loaded!
99 loaded!
983 loaded!
loading pretrained model: /content/drive/MyDrive/Colab Notebooks/GNN/MoTSE-main/src/../saved_models/PCBA/GCN/raw_multi1000-2/PCBA-504332_PCBA-485364.pth
[0] training loss:0.5436626881361007
val roc_auc:0.5814238952536825
val auprc:0.5889425275003866
[20] training loss:0.5187508022785187
val roc_auc:0.5953355155482816
val auprc:0.5913257873422961
test roc_auc:0.6091103362019696
test auprc:0.570640087596554
ft on PCBA-504333


<ipython-input-20-2d08720ff282>:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(result, ignore_index=True)


770 loaded!
91 loaded!
954 loaded!
loading pretrained model: /content/drive/MyDrive/Colab Notebooks/GNN/MoTSE-main/src/../saved_models/PCBA/GCN/raw_multi1000-2/PCBA-504333_PCBA-1460.pth
[0] training loss:0.5849057305604219
val roc_auc:0.6497560975609756
val auprc:0.690802624464947
[20] training loss:0.5526255294680595
val roc_auc:0.5829268292682926
val auprc:0.6245790301877288
test roc_auc:0.6643089502373833
test auprc:0.6733136580887564
Results have been saved to /content/drive/MyDrive/Colab Notebooks/GNN/MoTSE-main/src/../results/PCBA/GCN/multi1000-2/results.csv


<ipython-input-20-2d08720ff282>:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(result, ignore_index=True)


In [ ]:
#Pre-train GNN w/ target task and third source task, using pre-trained weights from target task + first two source tasks
#Use target data to fine-tune model pre-trained on 2 source tasks + target

multi_train(source_target_pairs, 3, second_ft_model_path, third_train_model_path)
multi_ft(source_target_pairs, 3, third_train_model_path, third_ft_model_path)

multi PCBA-1030 PCBA-1460
819 loaded!
100 loaded!
980 loaded!
816 loaded!
96 loaded!
970 loaded!
loading pretrained model: /content/drive/MyDrive/Colab Notebooks/GNN/MoTSE-main/src/../saved_models/PCBA/GCN/multi1000-2/PCBA-1030.pth
initializing trainer...
[0] training loss:1.2183327885235058
PCBA-1030
val roc_auc:0.685874349739896
val auprc:0.6944182909336133
PCBA-1460
val roc_auc:0.596078431372549
val auprc:0.5708252885120724
[20] training loss:1.1053031647906584
PCBA-1030
val roc_auc:0.6482593037214885
val auprc:0.642700951274669
PCBA-1460
val roc_auc:0.6583877995642702
val auprc:0.6540064341929017
[40] training loss:1.0503919253162308
PCBA-1030
val roc_auc:0.6770708283313326
val auprc:0.6896984343596914
PCBA-1460
val roc_auc:0.6644880174291938
val auprc:0.6755513943629359
test roc_auc:0.5939685514943247
test auprc:0.5620904906410924
test roc_auc:0.7069844278250179
test auprc:0.7058724708549476
multi PCBA-1458 PCBA-2546


<ipython-input-20-2d08720ff282>:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(result, ignore_index=True)


824 loaded!
92 loaded!
976 loaded!
767 loaded!
108 loaded!
955 loaded!
loading pretrained model: /content/drive/MyDrive/Colab Notebooks/GNN/MoTSE-main/src/../saved_models/PCBA/GCN/multi1000-2/PCBA-1458.pth
initializing trainer...
[0] training loss:1.2235792050970362
PCBA-1458
val roc_auc:0.8837429111531192
val auprc:0.8731594832372315
PCBA-2546
val roc_auc:0.5452314165497896
val auprc:0.6327684102656255
[20] training loss:1.0359062057860353
PCBA-1458
val roc_auc:0.8823251417769377
val auprc:0.849770025459169
PCBA-2546
val roc_auc:0.6637447405329593
val auprc:0.7408758397209746
[40] training loss:1.0009279948599794
PCBA-1458
val roc_auc:0.8790170132325141
val auprc:0.8529218542584046
PCBA-2546
val roc_auc:0.6767180925666199
val auprc:0.7537039578550084
[60] training loss:0.9591663657350743
PCBA-1458
val roc_auc:0.8818525519848772
val auprc:0.8628465087791017
PCBA-2546
val roc_auc:0.6896914446002805
val auprc:0.7625336312661152
[80] training loss:0.9265350280924046
PCBA-1458
val roc_auc:

<ipython-input-20-2d08720ff282>:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(result, ignore_index=True)


816 loaded!
96 loaded!
970 loaded!
767 loaded!
108 loaded!
955 loaded!
loading pretrained model: /content/drive/MyDrive/Colab Notebooks/GNN/MoTSE-main/src/../saved_models/PCBA/GCN/multi1000-2/PCBA-1460.pth
initializing trainer...
[0] training loss:1.1622017228857
PCBA-1460
val roc_auc:0.6832244008714597
val auprc:0.7299149550772013
PCBA-2546
val roc_auc:0.6553295932678822
val auprc:0.7050388021353766
[20] training loss:1.0475815204863852
PCBA-1460
val roc_auc:0.7363834422657952
val auprc:0.7719225474523674
PCBA-2546
val roc_auc:0.6605890603085555
val auprc:0.7299335895337442
[40] training loss:1.0142577290534973
PCBA-1460
val roc_auc:0.7450980392156864
val auprc:0.7697742939366734
PCBA-2546
val roc_auc:0.6795231416549788
val auprc:0.7500983973758478
[60] training loss:0.9848192828766843
PCBA-1460
val roc_auc:0.7098039215686274
val auprc:0.7509822001547566
PCBA-2546
val roc_auc:0.6802244039270687
val auprc:0.7495582714795969
test roc_auc:0.7078253594235563
test auprc:0.7085846724613925


...

<ipython-input-20-2d08720ff282>:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(result, ignore_index=True)


810 loaded!
99 loaded!
983 loaded!
loading pretrained model: /content/drive/MyDrive/Colab Notebooks/GNN/MoTSE-main/src/../saved_models/PCBA/GCN/raw_multi1000-3/PCBA-504332_PCBA-1460.pth
[0] training loss:0.5204635322093963
val roc_auc:0.5842880523731587
val auprc:0.5856102604745465
[20] training loss:0.49197483420372007
val roc_auc:0.5977905073649754
val auprc:0.5870615628426665
test roc_auc:0.5990756309481409
test auprc:0.5646378510516444
ft on PCBA-504333


<ipython-input-20-2d08720ff282>:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(result, ignore_index=True)


770 loaded!
91 loaded!
954 loaded!
loading pretrained model: /content/drive/MyDrive/Colab Notebooks/GNN/MoTSE-main/src/../saved_models/PCBA/GCN/raw_multi1000-3/PCBA-504333_PCBA-2546.pth
[0] training loss:0.5771075605104367
val roc_auc:0.5868292682926829
val auprc:0.6268569422823919
[20] training loss:0.5605096208552519
val roc_auc:0.6253658536585366
val auprc:0.6837965700910105
test roc_auc:0.6693027958501847
test auprc:0.6785428095625168
Results have been saved to /content/drive/MyDrive/Colab Notebooks/GNN/MoTSE-main/src/../results/PCBA/GCN/multi1000-3/results.csv


<ipython-input-20-2d08720ff282>:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(result, ignore_index=True)


In [ ]:
#Coallate results

import pandas as pd
from utils import makedir

metric = dataset_metrics[0]
results_path = path_to_src + f"/../results/{dataset_name}/GCN/"
makedir(results_path + 'multi1000')

results_df1 = pd.read_csv(results_path + 'multi1000-1/results.csv')
results_df2 = pd.read_csv(results_path + 'multi1000-2/results.csv')
results_df3 = pd.read_csv(results_path + 'multi1000-3/results.csv')
results_df = pd.concat([results_df1, results_df2, results_df3])
results_df = results_df[['task', metric]]

idx = results_df.groupby(['task'])[metric].transform(max) == results_df[metric]
results_df = results_df[idx]
results_df = results_df.drop_duplicates(['task']) #If two source tasks have the same score, just choose one randomly
results_df.to_csv(results_path + 'multi1000/results.csv', index = False)